### Uncomment and run the following cells if you work on Google Colab :) Don't forget to change your runtime type to GPU!

In [33]:
# !git clone https://github.com/kstathou/vector_engine

In [34]:
# cd vector_engine

In [35]:
# pip install -r requirements.txt

### Let's begin!

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
# Used to import data from S3.
import pandas as pd
# import s3fs

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

# Used to do vector searches and display the results.
from vector_engine.utils import vector_search, id2details

pd.read_json("data/posts.json")

# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

# Convert abstracts to vectors
embeddings = model.encode(posts.title.to_list(), show_progress_bar=True)

# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, posts.id.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

# Define project base directory
# Change the index from 1 to 0 if you run this on Google Colab
project_dir = Path('notebooks').resolve().parents[1]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/models/stack_faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)
    

Stored and processed data in s3

In [3]:
# Use pandas to read files from S3 buckets!
df = pd.read_csv('s3://vector-search-blog/misinformation_papers.csv')

In [20]:
posts = pd.read_json("../data/answers.json")
posts.head()

,index,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,...,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,0,30571099,NaN,"<p>The best way I found to do it, is to genera...",NaN,NaN,0,NaN,1433154354770,NaN,...,1.433155e+12,None,3232601.0,None,3232601.0,27267951,2,0,NaN,NaN
1,1,30722980,NaN,<p>SQLPS will always return 0 if the command e...,NaN,NaN,0,NaN,1433823792770,NaN,...,NaN,None,NaN,None,1640611.0,27654548,2,0,NaN,NaN
2,2,30847183,NaN,<p>It is possible to assign a dynamic layout i...,NaN,NaN,0,NaN,1434376913367,NaN,...,NaN,None,NaN,None,1694735.0,27297648,2,0,NaN,NaN
3,3,30883342,NaN,<p>i have just started using html5 geolocation...,NaN,NaN,0,NaN,1434520761523,NaN,...,NaN,None,NaN,None,1521848.0,27523105,2,0,NaN,NaN
4,4,30599789,NaN,<p>Check the spelling of your stopwords. Malle...,NaN,NaN,0,NaN,1433257481253,NaN,...,1.444717e+12,None,2439905.0,None,2439905.0,27640825,2,1,NaN,NaN


In [22]:
pd.to_datetime(posts['creation_date'], unit='s')

OutOfBoundsDatetime: cannot convert input with unit 's'

In [4]:
df.head(3)

,original_title,abstract,year,citations,id,is_EN
0,When Corrections Fail: The Persistence of Poli...,An extensive literature addresses citizen igno...,2010,901,2132553681,1
1,A postmodern Pandora's box: anti-vaccination m...,The Internet plays a large role in disseminati...,2010,440,2117485795,1
2,Spread of (Mis)Information in Social Networks,We provide a model to investigate the tension ...,2010,278,2120015072,1


In [8]:
print(f"Misinformation, disinformation and fake news papers: {df.id.unique().shape[0]}")

Misinformation, disinformation and fake news papers: 8430


The [Sentence Transformers library](https://github.com/UKPLab/sentence-transformers) offers pretrained transformers that produce SOTA sentence embeddings. Checkout this [spreadsheet](https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/) with all the available models.

In this tutorial, we will use the `distilbert-base-nli-stsb-mean-tokens` model which has the best performance on Semantic Textual Similarity tasks among the DistilBERT versions. Moreover, although it's slightly worse than BERT, it is quite faster thanks to having a smaller size.

I use the same model in [Orion's semantic search engine](https://www.orion-search.org/)!

In [5]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

100%|██████████| 245M/245M [00:36<00:00, 6.72MB/s] 


cuda:0


In [6]:
next(model.parameters()).is_cuda

True

In [8]:
# Convert abstracts to vectors
embeddings = model.encode(posts.title.to_list(), show_progress_bar=True)

In [9]:
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')

Shape of the vectorised abstract: (768,)


## Vector similarity search with Faiss
[Faiss](https://github.com/facebookresearch/faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, even ones that do not fit in RAM. 
    
Faiss is built around the `Index` object which contains, and sometimes preprocesses, the searchable vectors. Faiss has a large collection of [indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes). You can even create [composite indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes-(composite)). Faiss handles collections of vectors of a fixed dimensionality d, typically a few 10s to 100s.

**Note**: Faiss uses only 32-bit floating point matrices. This means that you will have to change the data type of the input before building the index.

To learn more about Faiss, you can read their paper on [arXiv](https://arxiv.org/abs/1702.08734).

Here, we will the `IndexFlatL2` index:
- It's a simple index that performs a brute-force L2 distance search
- It scales linearly. It will work fine with our data but you might want to try [faster indexes](https://github.com/facebookresearch/faiss/wiki/Faster-search) if you work will millions of vectors.

To create an index with the `misinformation` abstract vectors, we will:
1. Change the data type of the abstract vectors to float32.
2. Build an index and pass it the dimension of the vectors it will operate on.
3. Pass the index to IndexIDMap, an object that enables us to provide a custom list of IDs for the indexed vectors.
4. Add the abstract vectors and their ID mapping to the index. In our case, we will map vectors to their paper IDs from MAG.

In [11]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, posts.id.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 10000


### Searching the index
The index we built will perform a k-nearest-neighbour search. We have to provide the number of neighbours to be returned. 

Let's query the index with an abstract from our dataset and retrieve the 10 most relevant documents. **The first one must be our query!**


In [13]:
# Paper abstract
posts.iloc[5415, 2]

'Apache oltu oauth2 ask token over SSL'

In [14]:
# Retrieve the 10 nearest neighbours
D, I = index.search(np.array([embeddings[5415]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [0.0, 126.71707153320312, 127.66409301757812, 141.100341796875, 142.92308044433594, 147.95599365234375, 148.1050567626953, 150.0005645751953, 151.20547485351562, 152.14971923828125]

MAG paper IDs: [27707907, 27713521, 27417842, 27605220, 27662868, 27517411, 27694543, 27419107, 27343463, 27737230]


In [16]:
# Fetch the paper titles based on their index
id2details(posts, I, 'title')

[['Apache oltu oauth2 ask token over SSL'],
 ['apache2 external disk permissions'],
 ['HtmlPurifierBundle How to configure URI.SafeIframeRegexp in config.yml Symfony2?'],
 ['Davical Sync-Token web request'],
 ['Meteor DDP SSL/Apache Proxy Connection'],
 ['apache 2 WWW redirection issue'],
 ['logging apache httpasyncclient'],
 ['Apache CXF - WS addressing how to set From, ReplyTo, Headers'],
 ['git server information or ascii on remote operations (clone)'],
 ['TokenResponseException - Error:"invalid_grant", Description:"", Uri:""']]

In [18]:
# Fetch the paper abstracts based on their index
id2details(posts, I, 'body')

[['<p>Is there a way to request an access token over SSL using Apache oltu?\nIt works great if I don\'t use https (port 8443) but just use http...</p>\n\n<p>The code I have:</p>\n\n<pre><code>OAuthClient oAuthClient = new OAuthClient(new URLConnectionClient());\n\nOAuthClientRequest request = OAuthClientRequest.tokenLocation(MessageFormat.format("https://{0}:8443/applicationId/oauth/token", host)) //\n        .setGrantType(GrantType.PASSWORD) //\n        .setUsername("username") //\n        .setPassword("password") //\n        .setClientId("clientId") //\n        .buildBodyMessage();\n\nOAuthAccessTokenResponse oAuthResponse = oAuthClient.accessToken(request);\n</code></pre>\n\n<p>I get following exception message:</p>\n\n<pre><code>    org.apache.oltu.oauth2.common.exception.OAuthSystemException: javax.net.ssl.SSLHandshakeException: java.security.cert.CertificateException: No name matching localhost found\nat org.apache.oltu.oauth2.client.URLConnectionClient.execute(URLConnectionClien


## Putting all together

So far, we've built a Faiss index using the misinformation abstract vectors we encoded with a sentence-DistilBERT model. That's helpful but in a real case scenario, we would have to work with unseen data. To query the index with an unseen query and retrieve its most relevant documents, we would have to do the following:

1. Encode the query with the same sentence-DistilBERT model we used for the rest of the abstract vectors.
2. Change its data type to float32.
3. Search the index with the encoded query.

Here, we will use the introduction of an article published on [HKS Misinformation Review](https://misinforeview.hks.harvard.edu/article/can-whatsapp-benefit-from-debunked-fact-checked-stories-to-reduce-misinformation/).


In [17]:
user_query = """
WhatsApp was alleged to have been widely used to spread misinformation and propaganda 
during the 2018 elections in Brazil and the 2019 elections in India. Due to the 
private encrypted nature of the messages on WhatsApp, it is hard to track the dissemination 
of misinformation at scale. In this work, using public WhatsApp data from Brazil and India, we 
observe that misinformation has been largely shared on WhatsApp public groups even after they 
were already fact-checked by popular fact-checking agencies. This represents a significant portion 
of misinformation spread in both Brazil and India in the groups analyzed. We posit that such 
misinformation content could be prevented if WhatsApp had a means to flag already fact-checked 
content. To this end, we propose an architecture that could be implemented by WhatsApp to counter 
such misinformation. Our proposal respects the current end-to-end encryption architecture on WhatsApp, 
thus protecting users’ privacy while providing an approach to detect the misinformation that benefits 
from fact-checking efforts.
"""

In [18]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [7.384466171264648, 57.3224983215332, 57.3224983215332, 71.48453521728516, 72.06803131103516, 79.13472747802734, 86.0128173828125, 89.91024780273438, 90.76014709472656, 90.76422119140625]

MAG paper IDs: [3047438096, 3021927925, 3037966274, 2889959140, 2791045616, 2943077655, 3014380170, 2967434249, 3028584171, 2990343632]


In [19]:
# Fetching the paper titles based on their index
id2details(df, I, 'original_title')

[['Can WhatsApp Benefit from Debunked Fact-Checked Stories to Reduce Misinformation?'],
 ['A Dataset of Fact-Checked Images Shared on WhatsApp During the Brazilian and Indian Elections'],
 ['A Dataset of Fact-Checked Images Shared on WhatsApp During the Brazilian and Indian Elections'],
 ['A System for Monitoring Public Political Groups in WhatsApp'],
 ['Politics of Fake News: How WhatsApp Became a Potent Propaganda Tool in India'],
 ['Characterizing Attention Cascades in WhatsApp Groups'],
 ['OS IMPACTOS JURÍDICOS E SOCIAIS DAS FAKE NEWS EM TERRITÓRIO BRASILEIRO'],
 ['Fake News and Social Media: Indian Perspective'],
 ['Images and Misinformation in Political Groups: Evidence from WhatsApp in India'],
 ['Can WhatsApp Counter Misinformation by Limiting Message Forwarding']]

In [19]:
# Define project base directory
# Change the index from 1 to 0 if you run this on Google Colab
project_dir = Path('notebooks').resolve().parents[1]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/models/stack_faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

/home/ryanwesslen/PycharmProjects/vector_engine
